# Problem #0

In [2]:
warnings.filterwarnings("ignore")

In [1]:
import warnings
import pandas as pd
import os
import numpy as np
import scattertext as st
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#import sys                                                    #Causes problems with regression in skikitlearn 
#reload(sys)                                                   #later on. Find different way to make Count Vectorizer
#sys.setdefaultencoding('utf8')                                #work. In future be carefull making global changes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import ARDRegression
from sklearn.ensemble import BaggingRegressor

C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [25]:
os.chdir(r'C:\Users\gwhit\Downloads\Stat517-master\Data')
salaryfn = 'salary_uk.csv'
salary_uk = pd.read_csv(salaryfn)

# Preprocessing

In [26]:
#drop redundant raw location and raw salary features
salary_uk = salary_uk.drop([u'LocationRaw', u'SalaryRaw', u'SourceName'],axis =1)

In [27]:
salary_uk.head()

,Id,Title,FullDescription,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryNormalized
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,25000
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,30000
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,30000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,27500
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000


In [28]:
salary_uk.columns

Index(['Id', 'Title', 'FullDescription', 'LocationNormalized', 'ContractType',
       'ContractTime', 'Company', 'Category', 'SalaryNormalized'],
      dtype='object')

In [29]:
#Uses CountVectorizer, I use TfidVectorizer which is superior to CountVectorizer because it 
#uses term frequency-inverse document frequency (TF–IDF) which weights the word counts by a measure 
#of how often they appear in the documents

#vec = CountVectorizer()
#X = vec.fit_transform(salary_uk['Title'].values.astype('string'))
#Title = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [30]:
#Vectorizes Title feature using term frequency-inverse document frequency 
vec = TfidfVectorizer()
X = vec.fit_transform(salary_uk['Title'].values.astype('str'))
Title = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [31]:
#Vectorizes FullDescription feature using term frequency-inverse document frequency 
vec = TfidfVectorizer()
X = vec.fit_transform(salary_uk['FullDescription'].values.astype('str'))
FullDescription = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [32]:
salary_uk = pd.merge(salary_uk, Title, left_index=True, right_index=True)

In [33]:
#This adds the FullDescription vector. It also makes the dataframe a little bit too big to run on my laptop so for now its
#commented out. Without this the datframe will have 3902 columns instead of 31087. The dataframe with 31087 columns causes
#memory error.

salary_uk =pd.merge(salary_uk, FullDescription, left_index=True, right_index=True)

In [34]:
salary_uk = salary_uk.drop(['Title','FullDescription'], axis = 1) #drops the now redundant columns

In [35]:
salary_uk = pd.get_dummies(salary_uk, columns =[u'LocationNormalized',
       u'ContractType', u'ContractTime', u'Company', u'Category'])

In [36]:
salary_uk.head()

,Id,SalaryNormalized,10_x,100_x,1000_x,11_x,11501250,12_x,140_x,15_x,...,Category_Other/General Jobs,"Category_PR, Advertising & Marketing Jobs",Category_Property Jobs,Category_Retail Jobs,Category_Sales Jobs,Category_Scientific & QA Jobs,Category_Social work Jobs,Category_Teaching Jobs,Category_Trade & Construction Jobs,Category_Travel Jobs
0,12612628,25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,12612830,30000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,12612844,30000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,12613049,27500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,12613647,25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# PCA

Although this problem doesn't explicitly call for a PCA I'm doing one because the analysis is prohibitively slow without it. 

In [37]:
Y_salary_uk = salary_uk['SalaryNormalized']
X_salary_uk = salary_uk.drop('SalaryNormalized', axis=1)

In [38]:
#Uses the first 1000 principle components for analysis
pca = PCA(n_components=1000)
pca.fit(X_salary_uk)
X_salary_uk = pca.transform(X_salary_uk)


Now we have our dataset ready for analysis.

# Partitioning the Data Set

In [40]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_salary_uk, Y_salary_uk,
                                                random_state=9)
Xcolnames = list(X_salary_uk)

In [41]:
Y_salary_uk.shape

(10000,)

In [42]:
X_salary_uk.shape

(10000, 1000)

# Exploratory Data Analysis

In [ ]:
Z = linkage(X_salary_uk, 'ward')
dn = dendrogram(Z)

# Analysis

# Linear Regression

In [43]:
lr = LinearRegression().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lr.score(Xtest, ytest)))

Training set score: 0.69
Test set score: 0.64


# Ridge Regression

In [44]:
ridge = Ridge().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ridge.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ridge.score(Xtest, ytest)))

Training set score: 0.69
Test set score: 0.64


C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.722842e-18
  overwrite_a=True).T


In [45]:
scores = cross_val_score(ridge, X_salary_uk, Y_salary_uk, cv=5)
print("Average cross-validation score: {:.2f}".format(scores.mean()))

C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.983795e-18
  overwrite_a=True).T
C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number6.770434e-19
  overwrite_a=True).T
C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.065165e-19
  overwrite_a=True).T
C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not g

Average cross-validation score: 0.51


C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.006234e-19
  overwrite_a=True).T


In [46]:
ridge10 = Ridge(alpha=10).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ridge10.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ridge10.score(Xtest, ytest)))

Training set score: 0.63
Test set score: 0.61


C:\Users\gwhit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.308541e-17
  overwrite_a=True).T


In [45]:
scores = cross_val_score(ridge10, X_salary_uk, Y_salary_uk, cv=5)
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: -2.65


# K Nearest Neighbors Regressor

In [46]:
# instantiate the model and set the number of neighbors to consider to 3:
reg = KNeighborsRegressor(n_neighbors=3)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.48


In [47]:
# instantiate the model and set the number of neighbors to consider to 50:
reg = KNeighborsRegressor(n_neighbors=50)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.31


In [48]:
# instantiate the model and set the number of neighbors to consider to 150:
reg = KNeighborsRegressor(n_neighbors=150)
# fit the model using the training data and training targets:
reg.fit(Xtrain, ytrain)
print("Test set R^2: {:.2f}".format(reg.score(Xtest, ytest)))

Test set R^2: 0.18


# Lasso

In [ ]:
lasso = Lasso().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso.coef_ != 0)))

In [34]:
scores = cross_val_score(lasso, X_salary_uk, Y_salary_uk, cv=5)
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.50


In [35]:
lasso001 = Lasso(alpha=0.9, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.68
Test set score: 0.64
Number of features used: 919


In [36]:
lasso01 = Lasso(alpha=0.1, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.68
Test set score: 0.64
Number of features used: 919


In [37]:
# we increase the default setting of "max_iter",
# otherwise the model would warn us that we should increase max_iter.
lasso001 = Lasso(alpha=0.01, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso001.coef_ != 0)))

Training set score: 0.68
Test set score: 0.63
Number of features used: 999


In [38]:
lasso00001 = Lasso(alpha=0.0001, max_iter=1000).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(lasso00001.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(lasso00001.score(Xtest, ytest)))
print("Number of features used: {}".format(np.sum(lasso00001.coef_ != 0)))

Training set score: 0.68
Test set score: 0.63
Number of features used: 1000


# Decision Tree Regressor

In [39]:
tree = DecisionTreeRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(tree.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(tree.score(Xtest, ytest)))

Training set score: 1.00
Test set score: 0.00


# Gradient Boosting Regressor

In [40]:
gbtr = GradientBoostingRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(gbtr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(gbtr.score(Xtest, ytest)))

Training set score: 0.64
Test set score: 0.50


# Ada Boost Regressor

In [41]:
abtr = AdaBoostRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(abtr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(abtr.score(Xtest, ytest)))

Training set score: -0.05
Test set score: -0.10


# Extra Trees Regressor

In [42]:
etr = ExtraTreesRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(etr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(etr.score(Xtest, ytest)))

Training set score: 1.00
Test set score: 0.51


# Random Forest Regressor

In [43]:
rfr = RandomForestRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(rfr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(rfr.score(Xtest, ytest)))

Training set score: 0.90
Test set score: 0.45


# Gaussian Process Regressor

In [ ]:
#takes a lil bit

gpr = GaussianProcessRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(gpr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(gpr.score(Xtest, ytest)))

# ARD Regression

In [ ]:
ardr = ARDRegression().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(ardr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(ardr.score(Xtest, ytest)))

# Bagging Regressor

In [ ]:
bingr = BaggingRegressor().fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(bingr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(bingr.score(Xtest, ytest)))

In [ ]:
bingr = BaggingRegressor(n_estimators = 5).fit(Xtrain, ytrain)
print("Training set score: {:.2f}".format(bingr.score(Xtrain, ytrain)))
print("Test set score: {:.2f}".format(bingr.score(Xtest, ytest)))

# Conclusion

Lasso with an alpha of .09 and max iteration of 1000 seemed to perform the best. Ridge regression with the standard hyperparameters also performed similarly which was interesting because Lasso and Ridge utilized a significantly different number of features. It also seemed strange to me that after doing a PCA linear regression performed much better than without a PCA. I suspect that this has something to do with how PCA works but I'm not exactly sure.